In [28]:
from graphviz import Digraph
# Process: Root gets activated by calling an activation task (defined and assigned to root runtime), root performs the task,
# saves the result in a 'context', calls the children's isActivationEligible by passing the context to the child,  
# every child that is eligible for activation does the same for its children. The path that is taken is relevant to the test data
# passed on the initial context. Once all test data is exhausted, all the un-visited nodes are shown (all nodes that haven't been visited
# after all test data is run, per test data obviously not all nodes will be visited because each test data will select a single flow
# to run) 

# Questions: Who should handle context - driver or node class. Used driver since context needs to be available across all nodes
# 2. How does the 'prior action results' object flow through the nodes? For now setting it on top node and having it flow through to children
# 3. How does the 'curr test data' object flow through the nodes? Same as above, but setting it on isActivationEligible

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v.children:
                edges.add((v, child))
                build(child)
    build(root)
    return nodes, edges

class Node:
    def __init__(self, description):
        self.description = description
        self.children = []
        self.testData = []
    
    def __repr__(self):
        return f"{self.description}"
    
    def setId(self, idd):
        self.id = idd

    def setContext(self, context):
        self.context = context
    
    def addChild(self, child):
        self.children.append(child)
    
    def display(self, globalVisited):
        dot = Digraph(format='svg', graph_attr={'rankdir': 'UD'}) # LR = left to right
        nodes, edges = trace(self)
        for n in nodes:
            uid = str(id(n))
            # for any value in the graph, create a rectangular ('record') node for it
            nodeColor = "green"
            if(n not in globalVisited):
                nodeColor = "red"
            dot.node(name = uid, label = "{ %s }" % (n), shape='record', color = nodeColor)

        for n1, n2 in edges:
            # connect n1 to the op node of n2
            dot.edge(str(id(n1)), str(id(n2)), n2.eligibilityDescription)

        return dot
    
    def setPriorActionResults(self, priorActionResults):
        self.priorActionResults = priorActionResults
    
    def setActivationEligibility(self, activationEligibilityFunction, eligibilityDescription):
        self.activationElibility=activationEligibilityFunction
        self.eligibilityDescription = eligibilityDescription
    
    def isActivationEligible(self, currTestData, context):
        isEligible = self.activationElibility(self.priorActionResults, currTestData, context)
        if(isEligible):
            self.setCurrTestData(currTestData)
        return isEligible
    
    def assignActivationTask(self, activationTaskFunction):
        self.activationTask=activationTaskFunction
        
    def setCurrTestData(self, currTestData):
        self.currTestData = currTestData
        if not currTestData in (self.testData):
            self.testData.append(currTestData)
    
    def activate(self, context, globalVisited):
        actionResult=self.activationTask(self.priorActionResults, self.currTestData, context)
        self.priorActionResults[self.description]=actionResult
        globalVisited.add(self)
#         print("action complete, actionResults: ", currActionResults)
        visited=set()
        
        for child in self.children:
            child.setPriorActionResults(self.priorActionResults)
            if((child.isActivationEligible(self.currTestData, context)) & (child not in visited)):
                visited.add(child)
                child.activate(context, globalVisited)
                
    def copyNodeWithoutChildren(self):
        nodeWithoutChildren = Node(self.description)
        nodeWithoutChildren.assignActivationTask(self.activationTask)
        nodeWithoutChildren.setActivationEligibility(self.activationElibility, self.eligibilityDescription)
        nodeWithoutChildren.testData = self.testData
        return nodeWithoutChildren
    
    def getIndividualBranches(self):
        if(len(self.children) == 0):
            nodeMinusChildren = self.copyNodeWithoutChildren()
            res = [[]]
            res[0].append(nodeMinusChildren)
            return res
        
        res = []
        for child in self.children:
            childBranches = child.getIndividualBranches()
            for childBranch in childBranches:
                childBranch.append(self.copyNodeWithoutChildren())
                res.append(childBranch)
        return res

In [29]:
class GlobalVisitedTestStub:
#     globalVisited = set()
    def add(self, nodeToRecord):
#         globalVisited.add(nodeToRecord.id)
        print(f'nodeDescription: {str(nodeToRecord.description)}, nodeTestData: {str(nodeToRecord.currTestData)}, nodeActionResults: {str(nodeToRecord.priorActionResults)}')

In [30]:
import requests

# productsApiNode
productsApiNode = Node("Call Products endpoint")
def productsApiNodeActivation(priorActionResults, currTestData, context):
    return True
productsApiNode.setActivationEligibility(productsApiNodeActivation, "Always true")
def productsApiNodeActivationTask(priorActionResults, currTestData, context):
    productsResponse = requests.get(context['baseUrl'] + "/products")
    return productsResponse
productsApiNode.assignActivationTask(productsApiNodeActivationTask)

#productIdApiNode
productIdApiNode = Node("Call Products Id endpoint")
def productIdApiNodeActivation(priorActionResults, currTestData, context):
    return True
productIdApiNode.setActivationEligibility(productIdApiNodeActivation, "Always true")
def productIdApiNodeActivationTask(priorActionResults, currTestData, context):
    productIdResponse = requests.get(context['baseUrl'] + "/products/" + str(currTestData))
    return productIdResponse
productIdApiNode.assignActivationTask(productIdApiNodeActivationTask)

#product1Node
product1Node = Node("Product1 Called")
def product1NodeActivation(priorActionResults, currTestData, context):
    return priorActionResults['Call Products Id endpoint'].json()['id'] == 1
product1Node.setActivationEligibility(product1NodeActivation, "Product1 Called")
def product1NodeActivationTask(priorActionResults, currTestData, context):
    print("Case 1 complete"); 
    return "Case 1 complete!"
product1Node.assignActivationTask(product1NodeActivationTask)

#product2Node
product2Node = Node("Product2 Called")
def product2NodeActivation(priorActionResults, currTestData, context):
    return priorActionResults['Call Products Id endpoint'].json()['id'] == 2
product2Node.setActivationEligibility(product2NodeActivation, "Product2 Called")
def product2NodeActivationTask(priorActionResults, currTestData, context):
    print("Case 2 complete"); 
    return "Case 2 complete!"
product2Node.assignActivationTask(product2NodeActivationTask)

productsApiNode.addChild(productIdApiNode)
productIdApiNode.addChild(product1Node)
productIdApiNode.addChild(product2Node)


In [31]:
globalVisitedTestObj = GlobalVisitedTestStub()

context = { "baseUrl": "https://fakestoreapi.com" }
testData = ["1", "2"]
for singleTestData in testData:
    productsApiNode.setPriorActionResults({})
    productsApiNode.isActivationEligible(singleTestData, context)
    productsApiNode.activate(context, globalVisitedTestObj)

nodeDescription: Call Products endpoint, nodeTestData: 1, nodeActionResults: {'Call Products endpoint': <Response [200]>}
nodeDescription: Call Products Id endpoint, nodeTestData: 1, nodeActionResults: {'Call Products endpoint': <Response [200]>, 'Call Products Id endpoint': <Response [200]>}
Case 1 complete
nodeDescription: Product1 Called, nodeTestData: 1, nodeActionResults: {'Call Products endpoint': <Response [200]>, 'Call Products Id endpoint': <Response [200]>, 'Product1 Called': 'Case 1 complete!'}
nodeDescription: Call Products endpoint, nodeTestData: 2, nodeActionResults: {'Call Products endpoint': <Response [200]>}
nodeDescription: Call Products Id endpoint, nodeTestData: 2, nodeActionResults: {'Call Products endpoint': <Response [200]>, 'Call Products Id endpoint': <Response [200]>}
Case 2 complete
nodeDescription: Product2 Called, nodeTestData: 2, nodeActionResults: {'Call Products endpoint': <Response [200]>, 'Call Products Id endpoint': <Response [200]>, 'Product2 Called'